Script to create a list of flows across all objects.

Common flow types (processType):
* Flow - also known as a screen flow, is a flowinvoked by end user action in the UI
* Workflow - represents an automation built with Process Builder
* AutoLaunchedFlow - runs in the background and doesn't require user interaction, includes schedule-triggered flows, record-triggered flow, platform event-triggered flow and flows that are triggered by another automation, such as an Apex class or another Flow
* Orchestrator - manages the execution sequence of multiple flows

This script targets Flows of the Workflow, AutolauchedFlow and Orchestrator processTypes that are effectively acting as database triggers.

In [23]:
import os
import pandas as pd
import xml.etree.ElementTree as ET

# Set the directory path; path below assumes that this project is in the same directory as the Salesforce project 
directory_path = '../projectName/force-app/main/default/flows/'
flow_definition_directory_path = '../projectName/force-app/main/default/flowDefinitions/'

# Create an empty DataFrame
df = pd.DataFrame(columns=['Name', 'ProcessType', 'Object' , 'TriggerType', 'ActiveVersion'])

# Loop through each XML file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".xml"):
        file_path = os.path.join(directory_path, filename)

        # Load the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Create new row for dataframe
        flow_data = {}
        flow_data['Name'] =  root.find('{http://soap.sforce.com/2006/04/metadata}label').text
        flow_data['ProcessType'] =  root.find('{http://soap.sforce.com/2006/04/metadata}processType').text
        
        # Add trigger details where the processType is Workflow
        if (flow_data['ProcessType'] == 'Workflow'):
            # Get sObject
            object_type = root.find('{http://soap.sforce.com/2006/04/metadata}processMetadataValues[{http://soap.sforce.com/2006/04/metadata}name="ObjectType"]/{http://soap.sforce.com/2006/04/metadata}value/{http://soap.sforce.com/2006/04/metadata}stringValue')
            flow_data['Object'] = object_type.text
            # Get TriggerType
            trigger_type = root.find('{http://soap.sforce.com/2006/04/metadata}processMetadataValues[{http://soap.sforce.com/2006/04/metadata}name="TriggerType"]/{http://soap.sforce.com/2006/04/metadata}value/{http://soap.sforce.com/2006/04/metadata}stringValue')
            flow_data['TriggerType'] = trigger_type.text
        
        # Add trigger details where the processType is Orchestrator or AutoLaunchedFlow
        if (flow_data['ProcessType'] == 'Orchestrator' OR flow_data['ProcessType'] == 'AutoLaunchedFlow'):
            # Get sObject
            object_type = root.find('{http://soap.sforce.com/2006/04/metadata}start/{http://soap.sforce.com/2006/04/metadata}object')
            flow_data['Object'] = object_type.text
            # Get TriggerType
            trigger_type = root.find('{http://soap.sforce.com/2006/04/metadata}start/{http://soap.sforce.com/2006/04/metadata}recordTriggerType')
            flow_data['TriggerType'] = trigger_type.text


        # Get the flow status
        definiton_filename = filename.split('.')[0] + '.flowDefinition-meta.xml'
        definiton_filepath = os.path.join(flow_definition_directory_path, definiton_filename)
        definiton_tree = ET.parse(definiton_filepath)
        definiton_root = definiton_tree.getroot()
        active_version = definiton_root.find('{http://soap.sforce.com/2006/04/metadata}activeVersionNumber')
        if active_version is not None:
            flow_data['ActiveVersion'] = active_version.text
        else:
            flow_data['ActiveVersion'] = "Inactive"

        # Append the data to the list
        df.loc[len(df)] = flow_data

# Display the DataFrame
print(df)

df.to_csv('flows.csv', index=False)

                                                  Name       ProcessType  \
0                                  New Contact Process          Workflow   
1        Convert Multiselect Picklist to List of Texts  AutoLaunchedFlow   
2    Customer Support Staff Submit a Case To Academ...              Flow   
3               Product Enrollment - Scheduled Actions  AutoLaunchedFlow   
4                        Promotional Code - After Save  AutoLaunchedFlow   
..                                                 ...               ...   
133                               2 Day Pause SA Email  AutoLaunchedFlow   
134                           Acad_RestartSubscription              Flow   
135    Summary of EC notifications report subscription  AutoLaunchedFlow   
136                               Add to Cohort Button              Flow   
137                               Multi Select Utility  AutoLaunchedFlow   

      Object   TriggerType ActiveVersion  
0    Contact  onAllChanges      Inactive  
1